<a href="https://colab.research.google.com/github/mattiapocci/PhilosoperRank/blob/master/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Imports and Drive Mount***

In [10]:

#!pip3 install -U spacy[cuda92]
#!python3 -m spacy download en_core_web_lg

import sys
import time
import os
import numpy as np
import random
import json
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
spacy.prefer_gpu()

from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#***Import JSON articles***


In [23]:
"""
{
    "philosopher": "name",
    "article": "plaintext_article",
    "pageid": "id"
}
"""
#open the file
with open('/content/drive/My Drive/folder/result.json') as f:
  jsonlist = json.load(f)
print(jsonlist[0]["article"])

Leo the Hebrew, pen name of Judá Abravanel (otherwise known as: in Latin, Leo Hebraeus; in Portuguese, Leão Hebreu; in Italian, Leone Ebreo; in Spanish, León Hebreo; in French, León l'Hebreu and in Hebrew, יְהוּדָה בֶּן יִצְחָק אַבְּרַבַנְאֵל [Yehuda ben Yitzhak Abravanel]) (c. 1460 Lisbon – c. 1530 ? Naples?), was a Portuguese Jewish philosopher, physician and poet. His work Dialogues of Love was one of the most important philosophical works of his time.


== Biography ==
The Abravanel (or Abrabanel) family was extraordinarily prominent among Jewish families in the Middle Ages, active in public service in the court of Castile. Judah (or Leon, as he is known in Spanish) was the son of Isaac ben Judah Abravanel, (meaning Isaac “son of Judah” Abravenel) who, according to Soria was “the last great commentator of the Bible of Medieval Jewry” (12). Don Isaac was a statesman, financier, and a defender of the Jewish faith, who was also born in Lisbon in 1437 and spent most of his life in Port

# ***Language Analysis***

In [0]:

"""
1.X cite(s|d) Y1, Y2, …, Yn as (an) influence(s)
2. X was influenced by Y1, Y2, …, Yn
3. Y has been cited as an influence by X1, X2, …, Xn
4. Y influence on X1, X2, …, Xn …
5. Y1, Y2, …, Yn influenced hi(m|s) …
"""

doc1 = nlp("Mark was influenced by Jhon")
doc2 = nlp("Jhon influenced Mark")
doc3 = nlp("Mark was an influence of Jhon and Albert")
doc4 = nlp("Jhon influenced him")
doc3 = nlp("Adler was also influenced by the philosophies of Immanuel Kant, Friedrich Nietzsche, Rudolf Virchow and the statesman Jan Smuts ")
doc3 = nlp("He influenced notable figures in subsequent schools of psychotherapy such as Rollo May, Viktor Frankl, Abraham Maslow and Albert Ellis ")
print("------")
print("1 Mark was influenced by Jhon")
displacy.render(doc1, style="dep", jupyter=True)
print("CHUNKS")
for chunk in doc1.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, 
            chunk.root.head.text)
print("TREE")
for token in doc1:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
    
print("------")
print("2 Jhon influenced Mark")
displacy.render(doc2, style="dep", jupyter=True)
print("CHUNKS")

for chunk in doc2.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, 
            chunk.root.head.text)
print("TREE")
for token in doc2:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
    
print("------")
print("3 Mark was an influence of Jhon and Albert")
displacy.render(doc3, style="dep", jupyter=True)
print("CHUNKS")
    
for chunk in doc3.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, 
            chunk.root.head.text)
print("TREE")
for token in doc3:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
    
print("------")
print("4 Jhon influenced him")
print("CHUNKS")
    
for chunk in doc4.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, 
            chunk.root.head.text)
print("TREE")
for token in doc4:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

    
#print(jsonlist[0]["article"])
#doc = nlp(jsonlist[0]["article"])

# ***Text Preprocessing***

In [0]:
#HIM
#divide in sentences the article

#eliminate sentences that don't include influenc -e, -ed, -es, etc

#pattern matching on patterns (see language analysis)

#Entity recognition and recostruction

#US
"mark influenced jhon. Today is friday"
#divide in sentences the article
"mark influenced jhon"
"Today is friday"
#eliminate sentences that don't include influenc -e, -ed, -es, etc
"mark influenced jhon"

#pattern matching on patterns (using spacy dependencies?)
[x = "mark", y = "jhon"]

#Entity recognition and recostruction(here we have a proble, when he had Hendrix he could )


# ***Processing***